In [ ]:
# %load metamaps.py
"""Provides methods for loading/saving metamaps"""
import struct
from enum import IntEnum
from keras.utils import Sequence
import numpy as np
import os
import random
import numbers

METAMAP_FILE_VERSION = 0x100

class TileType(IntEnum):
    """Tile types in a metamap"""
    UNREACHABLE = 0
    EMPTY = 1
    WALL = 2
    PUSHWALL = 3
    DOOR = 4

class EncodingDim(IntEnum):
    """Dimensions for the one-hot encoding of a metamap"""
    PLAYABLE = 0
    SOLID = 1
    PASSAGE = 2

TileTypeToEncodingDim = {
    TileType.UNREACHABLE: EncodingDim.SOLID,
    TileType.EMPTY: EncodingDim.PLAYABLE,
    TileType.WALL: EncodingDim.SOLID,
    TileType.PUSHWALL: EncodingDim.PASSAGE,
    TileType.DOOR: EncodingDim.PASSAGE,
    }

class MetaMapsSequence(Sequence):
    """A sequence of real metamaps from a directory and randomly generated ones"""

    def __init__(self, maps_dir, batch_size):
        self.maps_dir = maps_dir
        self.batch_size = batch_size
        self.map_files = os.listdir(maps_dir)
        NUM_MAPS = len(self.map_files)
        real_maps = [(index, True) for index in range(NUM_MAPS)]
        fake_maps = [(index + NUM_MAPS, False) for index in range(NUM_MAPS)]
        map_order = real_maps + fake_maps
        np.random.shuffle(map_order)
        self.map_order = map_order

    def __len__(self):
        return int(len(self.map_order) / self.batch_size)

    def __getitem__(self, idx):
        map_batch = np.zeros((self.batch_size, 64, 64, len(EncodingDim)))
        label_batch = np.zeros((self.batch_size))

        for i in range(self.batch_size):
            (index, real_map) = self.map_order[idx * self.batch_size + i]
            if real_map:
                label_batch[i] = 1
                map_batch[i:] = load_metamap(
                    os.path.join(self.maps_dir, self.map_files[index]))
            else:
                map_batch[i:] = generate_random_map()

        return map_batch, label_batch


def generate_random_map():
    """Generate a random map"""
    width = 64
    height = 64
    size = width * height

    junk_map = np.zeros([size, len(EncodingDim)])

    for i in range(size):
        tile_type = random.randint(0, len(EncodingDim) - 1)
        junk_map[i, tile_type] = 1

    junk_map.shape = (width, height, len(EncodingDim))

    return junk_map


def load_smashed_metamap(filename):
    """Loads a condensed metamap file"""
    with open(filename, "rb") as fin:
        mega_meta_map = np.fromfile(fin, dtype=np.uint8)
    return mega_meta_map


def load_all_metamaps(dirname, number_cap=None):
    """Loads all the metamaps in the given directory, returning a giant numpy array"""
    map_names = os.listdir(dirname)
    if isinstance(number_cap, numbers.Integral) and number_cap > 0:
        map_names = map_names[:number_cap]
    all_maps = np.zeros((len(map_names), 64, 64, len(EncodingDim)))

    for index, map_name in enumerate(map_names):
        load_metamap_into(os.path.join(dirname, map_name), all_maps, index)

    return all_maps


def load_metamap_into(filename, all_maps, index):
    """Loads a metamap from a file into a numpy array of shape (width, height, 3)"""
    with open(filename, "rb") as fin:
        version = struct.unpack('Q', fin.read(8))[0]

        if version != METAMAP_FILE_VERSION:
            raise ValueError("Unsupported metamap version")

        width = struct.unpack('i', fin.read(4))[0]
        height = struct.unpack('i', fin.read(4))[0]

        raw_map = np.fromfile(fin, dtype=np.uint8)
        raw_map.shape = (width, height)

        for y in range(height):
            for x in range(width):
                tile_type = TileType(raw_map[y, x])
                all_maps[index, y, x, TileTypeToEncodingDim[tile_type]] = 1


def load_metamap(filename):
    """Loads a metamap from a file into a numpy array of shape (width, height, 3)"""
    with open(filename, "rb") as fin:
        version = struct.unpack('Q', fin.read(8))[0]

        if version != METAMAP_FILE_VERSION:
            raise ValueError("Unsupported metamap version")

        width = struct.unpack('i', fin.read(4))[0]
        height = struct.unpack('i', fin.read(4))[0]
        size = width * height

        raw_map = np.fromfile(fin, dtype=np.uint8)
        one_hot = np.zeros([size, len(EncodingDim)])

        for i in range(size):
            tile_type = TileType(raw_map[i])
            one_hot[i, TileTypeToEncodingDim[tile_type]] = 1

        one_hot.shape = (width, height, len(EncodingDim))

        return one_hot


def save_metamap(metamap, filename):
    """Saves a metamap to a file"""
    with open(filename, "wb") as fout:
        fout.write(struct.pack('Q', METAMAP_FILE_VERSION))

        width = metamap.shape[0]
        height = metamap.shape[1]

        fout.write(struct.pack('i', width))
        fout.write(struct.pack('i', height))
        for y in range(height):
            for x in range(width):
                tile_type = TileType.WALL
                if metamap[y, x, EncodingDim.PLAYABLE] == 1:
                    tile_type = TileType.EMPTY
                elif metamap[y, x, EncodingDim.SOLID] == 1:
                    tile_type = TileType.WALL
                elif metamap[y, x, EncodingDim.PASSAGE] == 1:
                    tile_type = TileType.DOOR

                fout.write(struct.pack('b', tile_type))
    return

def save_map_as_text(encoded_map, filename):
    """Saves a numpy array of size 64x64x3 representing an encoded Wolf map as a text file"""
    with open(filename, "w") as fout:
        for y in range(64):
            for x in range(64):
                if encoded_map[y, x, EncodingDim.PLAYABLE] == 1:
                    tile = ' '
                elif encoded_map[y, x, EncodingDim.SOLID] == 1:
                    tile = '#'
                elif encoded_map[y, x, EncodingDim.PASSAGE] == 1:
                    tile = '+'

                fout.write(tile)
            fout.write('\n')

    return

In [ ]:
mega = load_smashed_metamap("MegaMetaMapCondensed").astype('bool')
print(f"Mega shape: {mega.shape}")

In [ ]:
WIDTH = 64
HEIGHT = 64
ALPHABET = len(EncodingDim)

In [ ]:
mega.shape = (int(mega.shape[0]/ALPHABET), ALPHABET)
print(f"Mega reshaped: {mega.shape}")

In [ ]:
num_rows = int(mega.shape[0]/WIDTH)
print(f"Number of rows: {num_rows:,d}")

In [ ]:
sampling_step = 4
total_samples = int((mega.shape[0] - WIDTH - 1) / sampling_step)

print(f"Total samples: {total_samples:,d}")

# Vectorizing

In [ ]:
samples = np.zeros((total_samples, WIDTH, ALPHABET), dtype=np.bool)
next_tiles = np.zeros((total_samples, ALPHABET), dtype=np.bool)

In [ ]:
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [ ]:
samples_byte_size = samples.size * samples.dtype.itemsize
next_tiles_byte_size = next_tiles.size * next_tiles.dtype.itemsize
print(f"Samples size: {sizeof_fmt(samples_byte_size)}")
print(f"Next Tiles size: {sizeof_fmt(next_tiles_byte_size)}")
print(f"Total: {sizeof_fmt(samples_byte_size + next_tiles_byte_size)}")

In [ ]:
for sample_index in range(total_samples):
    start_offset = sample_index * sampling_step
    samples[sample_index] = mega[start_offset: start_offset + WIDTH]
    next_tiles[sample_index] = mega[start_offset + WIDTH]

# Create Model

In [ ]:
import keras
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(WIDTH, ALPHABET)))
model.add(layers.Dense(ALPHABET, activation='softmax'))

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Training and Generation

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [2]:
import os
import shutil

output_dir = "tiledriver_ltsm"
shutil.rmtree(output_dir, ignore_errors=True)
os.mkdir(output_dir)

In [ ]:
import datetime
import random
import sys

print(f"Started: {datetime.datetime.time(datetime.datetime.now())}")

EPOCHS = 60

for epoch in range(0, EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    model.fit(samples, next_tiles, batch_size=128, epochs=1)
    start_row_index = random.randint(0, num_rows) * WIDTH

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        generated_map = np.zeros((HEIGHT*WIDTH,ALPHABET))
        generated_map[0:WIDTH] = mega[start_row_index: start_row_index + WIDTH]      
        for i in range((HEIGHT-1)*WIDTH):
            sampled = generated_map[i:i+WIDTH]
            preds = model.predict(sampled, verbose=0)[0]
            next_tile = sample(preds, temperature)
            generated_map[WIDTH + i, next_tile] = 1
        generated_map.shape = (HEIGHT,WIDTH,ALPHABET)
        save_map_as_text(generated_map, os.path.join(output_dir, f"epoch_{epoch:02}_t{temperature}.txt")

print(f"Done: {datetime.datetime.time(datetime.datetime.now())}")